In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time
import random

In [2]:
books = pd.read_csv('books.csv')
ratings = pd.read_csv('ratings.csv')
users = pd.read_csv('users.csv')

C:\Users\zmajernik\AppData\Local\Temp\ipykernel_19752\835546400.py:1: DtypeWarning: Columns (3,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('books.csv')


In [3]:
books = books[['ISBN', 'Book-Title', 'Book-Author']]
books.head()

,ISBN,Book-Title,Book-Author
0,195153448,Classical Mythology,Mark P. O. Morford
1,2005018,Clara Callan,Richard Bruce Wright
2,60973129,Decision in Normandy,Carlo D'Este
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,393045218,The Mummies of Urumchi,E. J. W. Barber


In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
users.loc[users.loc[:,'User-ID'] == 244]

,User-ID,Location,Age
243,244,"college station, texas, usa",NaN


In [7]:
def findModes(lst, n):
    counter = Counter(lst)
    modes = counter.most_common(n)
    return [mode[0] for mode in modes]

In [17]:
numTopBooks = 6
modes = findModes(ratings['ISBN'], numTopBooks)
print(modes)
filteredRatings = ratings[ratings['ISBN'].isin(modes)].copy().reset_index(drop=True)
filteredRatings.head()

['971880107', '316666343', '385504209', '60928336', '312195516', '044023722X']


,User-ID,ISBN,Book-Rating
0,276925,316666343,0
1,276925,385504209,8
2,276925,971880107,0
3,276939,971880107,0
4,276954,971880107,0


In [18]:
filteredRatings = filteredRatings.drop_duplicates()
filteredRatings = filteredRatings.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating', aggfunc='mean')
filteredRatings.fillna(0, inplace=True)

In [22]:
scores = {}
clusters = len(filteredRatings)
for i in range(2, clusters):
    agg = AgglomerativeClustering(n_clusters=i, affinity='euclidean', linkage='ward')
    agg = agg.fit(filteredRatings)
    ss_avg = silhouette_score(filteredRatings, agg.labels_)
    scores[i] = ss_avg
    percentDone = round(((i-1)/(clusters-2))*100,2)
    clear_output(wait=True)
    time.sleep(0.001)
    print(str(percentDone) + "% done")

0.5% done


KeyboardInterrupt: 

In [ ]:
sns.scatterplot(x=scores.keys(), y=scores.values())
bestClusters = max(scores, key=scores.get)
print(bestClusters)